<a href="https://colab.research.google.com/github/yangsoyoung10011001/HumanActivityRecognition/blob/main/ConvLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow.keras

ERROR: Could not find a version that satisfies the requirement tensorflow.keras (from versions: none)
ERROR: No matching distribution found for tensorflow.keras


In [ ]:
# lstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling1D
from matplotlib import pyplot

In [ ]:
#load a single file as a numpy array

def load_file(filepath):
  dataframe = read_csv(filepath, header = None, delim_whitespace = True)
  return dataframe.values



In [ ]:
# load a list of files into a 3D array of [samples, timesteps, features]
#samples
def load_group(filenames, prefix=''):
  loaded = list()
  for name in filenames:
    data = load_file(prefix +name)
    loaded.append(data)
    #stack group so that features are the 3rd dimension
  loaded = dstack(loaded)
  return loaded
  #  there are 128 time steps and nine features, where the number of samples is the number of rows in any given raw signal data file.

In [ ]:
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
  filepath = prefix + group+'/Inertial Signals/'
  #load all 9 files as a single array
  filenames = list()
  #total acceleration
  filenames += ['total_acc_x_' +group +'.txt', 'total_acc_y_'+group+'.txt','total_acc_z_'+group+'.txt']
  #body accelation
  filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
  #body gyroscope
  filenames +=['body_gyro_x_' +group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
  #load input data
  X=load_group(filenames, filepath)
  #load class output
  y=load_file(prefix + group+ '/y_'+group+'.txt')
  return X,y


In [ ]:
#load the dataset, returns train and test X and yelements
def load_dataset(prefix='/content/drive/MyDrive/오영민교수님연구실/UCI HAR Dataset/UCI '):
  #/content/drive/MyDrive/오영민교수님연구실/UCI HAR Dataset/UCI HAR Dataset/test/Inertial Signals
  #load all train
  trainX, trainy = load_dataset_group('train', prefix+'HAR Dataset/')
  print(trainX.shape, trainy.shape)
  #load all test
  testX, testy = load_dataset_group('test',prefix+'HAR Dataset/')
  print(testX.shape, testy.shape)
  #zero-offset class values
  trainy = trainy - 1
  testy = testy -1
  #one hot encode y
  trainy = to_categorical(trainy)
  testy = to_categorical(testy)
  print(trainX.shape, trainy.shape, testX.shape, testy.shape)
  return trainX, trainy, testX, testy


In [ ]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 0, 25, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	#reshape data into time steps of sub-sequences
	n_steps, n_length = 4, 32
	trainX = trainX.reshape((trainX.shape[0], n_steps,1, n_length, n_features))
	testX = testX.reshape((testX.shape[0], n_steps, 1,n_length, n_features))
	#define model
	model = Sequential()
	model.add(Conv2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps,1, n_length,n_features)))
	model.add(Dropout(0.5))
	model.add(Flatten())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs,activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
	# model.add(TimeDistributed(Flatten()))
	# model.add(LSTM(100))
	# model.add(Dropout(0.5))
	
	# # model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
	# # model.add(Dropout(0.5))
	# model.add(Dense(100, activation='relu'))
	
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy
  #model is fit fixed epoch(in this case 15), a batch size of 64 samples 
  #-> 64 windows of data will be exposed to the model before the weights of the model are updated



In [ ]:


# summarize scores
def summarize_results(scores):
  print(scores)
  m,s = mean(scores), std(scores)
  print('Accuracy: %.3f%% (+/-%.3f)' % (m,s))

#run an experiment
def run_experiment(repeats = 10):
  #load data
  trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
  scores = list()
  for r in range(repeats):
    score = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('#%d: %.3f' % (r+1,score))
    scores.append(score)
    #summarize results
  summarize_results(scores)

  #run the experiment
run_experiment()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
#1: 91.279
#2: 92.195
#3: 89.990
#4: 91.754
#5: 91.177
#6: 89.311
#7: 92.229
#8: 91.110
#9: 87.343
#10: 91.042
[91.27926826477051, 92.19545125961304, 89.98982310295105, 91.75432920455933, 91.17746949195862, 89.31116461753845, 92.22938418388367, 91.10960364341736, 87.3430609703064, 91.0417377948761]
Accuracy: 90.743% (+/-1.422)
